In [1]:
import torch
from models import BiFPN_Concat2
from models import BiFPN_Concat3

In [2]:
# 1. 模拟特征图输入 (Batch_size, Channels, Height, Width)
    # 假设通道数为 256，尺寸为 40x40
feat1 = torch.randn(1, 256, 40, 40)
feat2 = torch.randn(1, 256, 40, 40)
feat3 = torch.randn(1, 256, 40, 40)

# 2. 实例化模块
bifpn2 = BiFPN_Concat2(dimension=1)
bifpn3 = BiFPN_Concat3(dimension=1)

# 3. 前向传播测试
out2 = bifpn2([feat1, feat2])
out3 = bifpn3([feat1, feat2, feat3])

print(f"输入尺寸: {feat1.shape}")
print(f"BiFPN_Concat2 输出尺寸: {out2.shape} (预期通道: 512)")
print(f"BiFPN_Concat3 输出尺寸: {out3.shape} (预期通道: 768)")

# 验证输出维度
assert out2.shape[1] == 512, "BiFPN_Concat2 通道融合错误"
assert out3.shape[1] == 768, "BiFPN_Concat3 通道融合错误"

# 4. 反向传播测试 (验证权重是否可学习)
print("\n=== 开始可学习权重测试 ===")
optimizer = torch.optim.SGD(bifpn3.parameters(), lr=0.1)

initial_weights = bifpn3.w.detach().clone()
print(f"初始权重: {initial_weights.numpy()}")

# 模拟一次损失计算和反向传播
loss = out3.sum()
loss.backward()
optimizer.step()

updated_weights = bifpn3.w.detach().clone()
print(f"更新后权重: {updated_weights.numpy()}")

if not torch.equal(initial_weights, updated_weights):
    print("✅ 权重已成功更新，模块具有学习能力。")
else:
    print("❌ 权重未变化，请检查 requires_grad 设置。")

# 5. ONNX 导出兼容性测试
# print("\n=== 开始 ONNX 导出测试 ===")
# try:
#     dummy_input = [torch.randn(1, 256, 20, 20) for _ in range(3)]
#     torch.onnx.export(bifpn3, (dummy_input,), "bifpn_test.onnx", opset_version=12)
#     print("✅ ONNX 导出成功，模块兼容部署。")
# except Exception as e:
#     print(f"❌ ONNX 导出失败: {e}")

输入尺寸: torch.Size([1, 256, 40, 40])
BiFPN_Concat2 输出尺寸: torch.Size([1, 512, 40, 40]) (预期通道: 512)
BiFPN_Concat3 输出尺寸: torch.Size([1, 768, 40, 40]) (预期通道: 768)

=== 开始可学习权重测试 ===
初始权重: [1. 1. 1.]
更新后权重: [ 18.342499    0.7034523 -16.04494  ]
✅ 权重已成功更新，模块具有学习能力。
